In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

Load data

In [ ]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize( (0.5,0.5,0.5), (0.5,0.5,0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

trainloader=torch.utils.data.DataLoader(trainset, batch_size=8, shuffle=True, num_workers=2)
#trainloader = torch.DataLoader(trainset,batch_size=8, shuffle=True, num_workers=2) 
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

testloader=torch.utils.data.DataLoader(testset, batch_size=8, shuffle=False, num_workers=2)

classes=('plane','car','bird','cat','deer','dog','frog','horse','ship','truck')

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


#Build a model

init에서 i개의 layer를 가진 convnet을 생성하고 forward에서 i번에 따른 연산을 진행 

In [ ]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module) :
  #nn.Module을 상속받음
  def __init__(self) :
    super(Net, self).__init__()
    #super().__init__ : Return a proxy object which represents the parent's class.
    #init부분에서 내가 원하는 연산을 코딩
    self.conv1 = nn.Conv2d(3,6,5)
    self.pool = nn.MaxPool2d(2,2)
    self.conv2 = nn.Conv2d(6,16,5)
    self.fc1 = nn.Linear(16*5*5,120)
    self.fc2 = nn.Linear(120,84)
    self.fc3 = nn.Linear(84,10)
    #class가 10개이기 때문에 output또한 10개

  def forward(self,x) :
    #연산의 순서를 정하는 part
    x= self.pool(F.relu(self.conv1(x)))
    #conv1에 대해 활성화 함수 relu 적용 후 maxpooling진행
    x= self.pool(F.relu(self.conv2(x)))
    x= x.view(-1,16*5*5)
    #FC를 위해 이미지를 1d로 바꾸는 과정
    x= F.relu(self.fc1(x))
    x= F.relu(self.fc2(x))
    x= self.fc3(x)
    return x

net = Net()
#class에 대한 instance

In [ ]:
print(net)

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


supervised learning에서는 output이 나오고 그 output을 바탕으로 loss, gradient를 구한 후 parameter를 업데이트 

#Implement the model with training data

Optimizer

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#SGD : StochasticGradient
#net.parameters() : net의 parameter를 반환
#lr은 scheduling을 통해 가변적으로 운영 가능 

실제적 학습 part

현재 dataloader에 8개의 batch만 존재하기 때문에 학습은 8개로만 진행됨

In [ ]:
for epoch in range(1) :   #loop over the dataset multiple times
#실제 데이터 연산은 더 많은 epoch를 반복
  running_loss=0.0
  max_loss = []
  for i, data in enumerate(trainloader,0) :
    #get the inputs ; data is a list of [inputs, labels]
    inputs, labels = data

    #zero the parameter gradient
    #optimize reset
    optimizer.zero_grad()

    #forward + backword + optimize
    outputs = net(inputs)
    loss = criterion(outputs, labels)
    #output에 대한 loss function
    loss.backward()
    optimizer.step()

    #print statistics
    running_loss += loss.item()
    max_loss.append(loss.item())
    #loss의 값을 하나의 숫자로 변환 후 update 
    if i % 2000 == 1999 :
      print('[%d, %5d] loss : %.3f'% (epoch +1 , i+1, running_loss / 2000))
      running_loss = 0.0
    # if i>2 :
    #   if max_loss[i] > max_loss[i-1] :
    #     break

    
    

print('Finished Training')
print('max_loss is : {}'.format(max_loss[-1]))

[1,  2000] loss : 2.304
[1,  4000] loss : 2.305
[1,  6000] loss : 2.304
Finished Training
max_loss is : 2.291095018386841


학습을 통한 loss의 값이 감소했다는 것은 학습이 제대로 이루어졌다로 생각해도됨

epoch가 진행됨에 따라 중간에 best model이 나올 가능성도 있음. 이 경우는 이를 끊어줄 수 있는 조건문을 생성하는 것도 가능 

#Save the Trained model

In [ ]:
#create a path
path = './cifar_net.pth'
torch.save(net.state_dict(), path)

#Load the pre-trained model

In [ ]:
net=Net()
net.load_state_dict(torch.load(path))

<All keys matched successfully>

output은 미니배치의 결과가 산출되기 때문에 for문을 통해 test 전체의 예측값을 구해야 함

In [ ]:
correct = 0
total = 0
with torch.no_grad() :
  #업데이트를 하지 않음 
  for data in testloader :
    images, labels = data
    outputs = net(images)
    #10개의 값이 존재하는 vector 
    _, predicted = torch.max(outputs.data, 1)
    #output에서 가장 큰 class의 결과값을 기준
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy of the network on the 10000 test images : %d %%' % (100*correct / total))

Accuracy of the network on the 10000 test images : 11 %
